<a href="https://colab.research.google.com/github/GabHoo/Challenging-SRL/blob/main/Test_Models.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# SETTING UP
Run the following cells in the Settin Up section to be able to run any of the tests in this notebook with the two suggested models.  
Change the current value for both model path according to your own models location. If No models are found they will be downloaded

In [48]:
# INSTALL AND IMPORTS
"""
pip install allennlp
pip install allennlp-models
pip install -U spaCy
pip install checklist
"""
from allennlp.predictors import Predictor
import allennlp_models.tagging

import json
import os
from utils import *
import utils
import re


### Change the model here:

In [17]:
#LOADING MODEL
model="Bilstm"

if model=="Bert":
    model_name="structured-prediction-srl-bert.2020.12.15.tar.gz"
    path_model="models/"+model_name
elif model=="Bilstm":
    model_name="openie-model.2020.03.26.tar.gz"
    path_model="models/"+model_name
else:
    print("Model not found!")
    exit()

if os.path.exists(path_model):
    print("Model found!")
    predictor = Predictor.from_path(path_model)
else:
    predictor = Predictor.from_path("https://storage.googleapis.com/"+model_name)

#TESTING IF THE MODELS ARE LOADED CORRECTLY

pred=predictor.predict("SRL model was loaded succeffully!")
if pred :
    print((" ").join(pred['words'])) 
    

Model found!


/home/gabhoo/.local/lib/python3.10/site-packages/spacy/util.py:887: UserWarning: [W095] Model 'en_core_web_sm' (3.3.0) was trained with spaCy v3.3 and may not be 100% compatible with the current version (3.5.1). If you see errors or degraded performance, download a newer compatible model or retrain your custom model with the current spaCy version. For more details and available updates, run: python -m spacy validate
  warnings.warn(warn_msg)


SRL model was loaded succeffully !


# Useful functions


In [3]:
def append_to_results(results_path,your_result):
    """Add results to the score board. 
    your_Results neets to be a dictionary alraedy'
    """
    if type(your_result) != dict:
        print("your_result needs to be a dictionary!")
        return
    
    with open(results_path, 'r') as f:
        data = json.load(f)
    
    data.update(your_result)

    with open(results_path, 'w') as f:
        json.dump(data, f,indent=4)


# PREDICATE IDENTIFICATION

In [31]:
#In this folder we have the data for the predicate classidication task. Change the path accordingly if files were moved.
path="./Data/Predicate_identification/"
results_path=f"./results_{model}_Predicate_Identification.json"
## we initialize also the dictionary that will contain the results
with open(results_path, 'w') as f:
    json.dump({"test":"failure_rate"},f)

## VOCABULARY+POS

### Test for contractions

In [34]:
data=json.load(open(path+'contracted_sentences.json'))
failure_rate=evaluate_PI_Contractions_INV(predictor,data)
print(f"Failure rate: {failure_rate}. Total number of tests: {len(data)}")
append_to_results(results_path,{"contracted_sentences":failure_rate})

Failure in I'd like some tea | I would like some tea
The alteration of verb 'd/would lead to a different labeling:

{'verb': "'d", 'description': "I 'd like some tea", 'tags': ['O', 'O', 'O', 'O', 'O']}
{'verb': 'would', 'description': '[ARG0: I] [V: would] [ARG1: like] [ARG1: some tea]', 'tags': ['B-ARG0', 'B-V', 'B-ARG1', 'B-ARG1', 'I-ARG1']}
Failure rate: 5.555555555555555. Total number of tests: 18


### Test for irregular inflections

In [36]:
data=json.load(open(path+"inflected_sentences.json"))
failure_rate=evaluate_PI_inflections_MFT(predictor,data)
print(f"Failure rate: {failure_rate}. Total number of tests: {len(data)}")
append_to_results(results_path,{"inflected_sentences":failure_rate})


Failed for: Gary vext an arm sleeve. did not detect vext

Failure rate: 2.857142857142857. Total number of tests: 35


## ROBUTSTNESS

In [37]:
data=json.load(open(path+"verb_typos_sentence.json"))
failure_rate=evaluate_PI_inflections_MFT(predictor,data)
print(f"Failure rate: {failure_rate}. Total number of tests: {len(data)}")
append_to_results(results_path,{"inflected_sentences":failure_rate})


Failed for: They aet a cool lot. did not detect aet

Failure rate: 1.8867924528301887. Total number of tests: 53


## AMBIGUITY

### Experiment with polysemic verbs [POLISEMIC]

In [38]:
data=json.load(open(path+'polysem_verbs_sentences.json'))
failure_rate=evaluate_PI_Polysem_DIR(predictor,data)
print(f"Failure rate: {failure_rate}. Total number of tests: {len(data)}")
append_to_results(results_path,{"polysemic_verbs":failure_rate})



Failed for: [I always turn left at the stop sign.],[She left her phone at home.]. left in both did NOT chage the predicate identification
['turn', 'left'] ['left']


Failed for: [I was lost],[I lost my phone]. lost in both did NOT chage the predicate identification
['was', 'lost'] ['lost']


Failure rate: 6.896551724137931. Total number of tests: 29


### Experiment with verbs being in different roles -ing [GERUNDS]

In [49]:
data=json.load(open(path+"gerunds.json"))
failure_rate=find_roleset_MFT(data,predictor)
print(f"Failure rate: {failure_rate}. Total number of tests: {len(data)}")
append_to_results(results_path,{"gerunds":failure_rate})

paitings not detected from From paintings of his done during that early period, only ['done'] were found ['done']
writing not detected from The writing of the thesis took a whole year., only ['took'] were found ['took']
singning not detected from The singing of the birds was a welcome sound in the morning., only ['was'] were found ['was']
reading not detected from What is your reading on the situation?, only ['is'] were found ['is']
cleaning not detected from The cleaning of the house was a tedious chore., only ['was'] were found ['was']
washing not detected from The washing of the dishes was a never-ending job., only ['was', 'ending'] were found ['was', 'ending']
cooking not detected from I have always enjoyed cooking, it is one of my favourite hobbies., only ['have', 'enjoyed', 'is'] were found ['have', 'enjoyed', 'is']
building not detected from The building of the bridge was a remarkable feat of engineering., only ['was'] were found ['was']
cutting not detected from The cutting of 

NameError: name 'append_to_results' is not defined

In [14]:
for s in sentences:
  print(s)
  pred=predictor.predict(s)
  
  verbs=[x['verb'] for x in pred['verbs']]
  #print(pred)
  print(verbs)
  #print(pred)

From paintings of his done during that early period
['done']
The writing on the wall was a warning of what was to come.
['was', 'was', 'come']
The singing of the birds was a welcome sound in the morning.
['was']
Singing in the shower is not allowed.
['Singing', 'is', 'allowed']
What is your reading on the situation?
['is']
The barking of the dog alerted us to the intruder.
['alerted']
The cleaning of the house was a tedious chore.
['was']
The washing of the dishes was a never-ending job.
['was', 'ending']
The cooking of the meal took longer than expected.
['took', 'expected']
The building of the bridge was a remarkable feat of engineering.
['was']
The cutting of the cake was the highlight of the party.
['was']
I have always enjoyed cooking, it is one of my favourite hobbies.
['have', 'enjoyed', 'is']
The protests that followed the killing of the priest were widespread.
['followed', 'were']
The sound of the river running was soothing to listen to.
['running', 'was', 'soothing', 'listen'

In [ ]:
for s in sentences:
  print(s)
  pred=predictor.predict(s)
  verbs=[x['verb'] for x in pred['verbs']]
  print(verbs)
  #print(pred)

In [ ]:
prediction=predictor.predict(
    sentence="I was writing a thesis"
)

In [ ]:
prediction

{'verbs': [{'verb': 'was',
   'description': 'I [V: was] writing a thesis',
   'tags': ['O', 'B-V', 'O', 'O', 'O']},
  {'verb': 'writing',
   'description': '[ARG0: I] was [V: writing] [ARG1: a thesis]',
   'tags': ['B-ARG0', 'O', 'B-V', 'B-ARG1', 'I-ARG1']}],
 'words': ['I', 'was', 'writing', 'a', 'thesis']}

In [ ]:
from nltk.corpus import propbank

[nltk_data] Downloading package propbank to /root/nltk_data...


True

In [ ]:
nltk.download('treebank')

[nltk_data] Downloading package treebank to /root/nltk_data...
[nltk_data]   Unzipping corpora/treebank.zip.


True

In [ ]:
sentences=["The birds singing in the morning was a welcome sound.","The singing of birds in the morning was a welcome sound."]
for s in sentences:
  print(s)
  predBert=predictor.predict(s)
  predBilstm=predictor_bilstm.predict(s)
  print("BERT")
  print([x['description'] for x in predBert['verbs']])

  print("Lstm")
  print([x['description'] for x in predBilstm['verbs']])
  
  print("\n\n")

The birds singing in the morning was a welcome sound.
BERT
['[ARG0: The birds] [V: singing] [ARGM-TMP: in the morning] was a welcome sound .', '[ARG1: The birds singing in the morning] [V: was] [ARG2: a welcome sound] .']
Lstm
['[ARG0: The birds] [V: singing] [ARGM-TMP: in the morning] was a welcome sound .', '[ARG1: The birds singing in the morning] [V: was] [ARG2: a welcome sound] .']



The singing of birds in the morning was a welcome sound.
BERT
['[ARG1: The singing of birds in the morning] [V: was] [ARG2: a welcome sound] .']
Lstm
['[ARG1: The singing of birds in the morning] [V: was] [ARG2: a welcome sound] .']





In [ ]:
sentences=["It's running low ","It is running low"]
for s in sentences:
  print(s)
  predBert=predictor.predict(s)
  predBilstm=predictor_bilstm.predict(s)
  print("BERT")
  print([x['description'] for x in predBert['verbs']])

  print("Lstm")
  print([x['description'] for x in predBilstm['verbs']])
  
  print("\n\n")

It's running low 
BERT
["It [V: 's] running low", "[ARG0: It] 's [V: running] [ARGM-MNR: low]"]
Lstm
["It [V: 's] running low", "[ARG1: It] 's [V: running] [ARG2: low]"]



It is running low
BERT
['It [V: is] running low', '[ARG1: It] is [V: running] [ARGM-MNR: low]']
Lstm
['[ARG1: It] [V: is] [ARG2: running low]', '[ARG1: It] is [V: running] [ARG2: low]']





## RARITY

In [ ]:
data=json.load(open(path+"verbs_slang.json"))
failure_rate=evaluate_PI_inflections_MFT(predictor,data)
print(f"Failure rate: {failure_rate}. Total number of tests: {len(data)}")
append_to_results(results_path,{"slang_verbs":failure_rate})


In [ ]:
data=json.load(open(path+"new_verbs.json"))
failure_rate=evaluate_PI_inflections_MFT(predictor,data)
print(f"Failure rate: {failure_rate}. Total number of tests: {len(data)}")
append_to_results(results_path,{"new_verbs":failure_rate})


# AROUGMENTS CLASSIFICATION

In [41]:
#In this folder we have the data for the predicate classidication task. Change the path accordingly if files were moved.
path="./Data/Argument_classification/"
results_path=f"./results_{model}_Argument_Classification.json"
## we initialize also the dictionary that will contain the results
with open(results_path, 'w') as f:
    json.dump({"test":"failure_rate"},f)

In [42]:

def eval_predictor_INV(sentences,predictor,function):
  """
  This function evaluates the predictor by using the function to validate the predictions.
  function is the name a function that takes two predictions and returns a boolean
  """
  fails=0
 
  f = getattr(utils, function)
  for pair in sentences:
    
    s1,s2=pair

    pred1 = predictor.predict(s1)
    pred2 = predictor.predict(s2) 
    
    match=f(pred1,pred2,verbose=False)
    
    if not (match):
      fails+=1

  return (fails/len(sentences)*100)

## VOCABULAIRTY+POS

### Pronouns vs Entity

In [ ]:
with open(path+"Pronouns_sents.json","r") as f:
    sentences=json.load(f)

sentences=list(sentences.items())

failure_rate=eval_predictor_INV(sentences,predictor,"validate_INV_allverbs_ARGset") #can try to do wit the othe function

print(f"Failure rate: {failure_rate}. Total number of tests: {len(sentences)}")

append_to_results(results_path,{f"Pronouns_sents":failure_rate})


## AMBIGUITY/TAXONOMY (PP-ATTACHMENT AMBIGUITY)

### PP-ATTACHMENT AMBIGUITY

In [ ]:
sentences="I went to the resturant by the Hutsin", "I went to the resturant by"
["I fixed the car with a red logo","I fixed the car with a wretch"]
["I bought a computer with GPU"," I bought a computer wit bitcoins"]

for s in sentences:
  print(s)
  pred=predictor.predict(s)
  desc=[x['description'] for x in pred['verbs']]
  #print(verbs)
  print(desc)
  print("\n\n")

In [28]:
pred=predictor_Bert.predict("I killed a rabbit with a knife")
pred

{'verbs': [{'verb': 'killed',
   'description': '[ARG0: I] [V: killed] [ARG1: a rabbit] [ARG2: with a knife]',
   'tags': ['B-ARG0',
    'B-V',
    'B-ARG1',
    'I-ARG1',
    'B-ARG2',
    'I-ARG2',
    'I-ARG2']}],
 'words': ['I', 'killed', 'a', 'rabbit', 'with', 'a', 'knife']}

In [40]:
pred=predictor_Bert.predict("Alice eats a chicken with a fork.")
pred

{'verbs': [{'verb': 'eats',
   'description': '[ARG0: Alice] [V: eats] [ARG1: a chicken] [ARGM-MNR: with a fork] .',
   'tags': ['B-ARG0',
    'B-V',
    'B-ARG1',
    'I-ARG1',
    'B-ARGM-MNR',
    'I-ARGM-MNR',
    'I-ARGM-MNR',
    'O']}],
 'words': ['Alice', 'eats', 'a', 'chicken', 'with', 'a', 'fork', '.']}

In [43]:
s=['Lukas eats a chicken with a feather.', 'Lukas eats a chicken with a stick.']
for x in s:
    pred=predictor_Bert.predict(x)
    print(pred)

{'verbs': [{'verb': 'eats', 'description': '[ARG0: Lukas] [V: eats] [ARG1: a chicken with a feather] .', 'tags': ['B-ARG0', 'B-V', 'B-ARG1', 'I-ARG1', 'I-ARG1', 'I-ARG1', 'I-ARG1', 'O']}], 'words': ['Lukas', 'eats', 'a', 'chicken', 'with', 'a', 'feather', '.']}
{'verbs': [{'verb': 'eats', 'description': '[ARG0: Lukas] [V: eats] [ARG1: a chicken] [ARGM-MNR: with a stick] .', 'tags': ['B-ARG0', 'B-V', 'B-ARG1', 'I-ARG1', 'B-ARGM-MNR', 'I-ARGM-MNR', 'I-ARGM-MNR', 'O']}], 'words': ['Lukas', 'eats', 'a', 'chicken', 'with', 'a', 'stick', '.']}


In [44]:
s=['Lukas eats a chicken with hands.', 'Lukas eats a chicken with olives.']
for x in s:
    pred=predictor_Bert.predict(x)
    print(pred)

{'verbs': [{'verb': 'eats', 'description': '[ARG0: Lukas] [V: eats] [ARG1: a chicken] [ARGM-MNR: with hands] .', 'tags': ['B-ARG0', 'B-V', 'B-ARG1', 'I-ARG1', 'B-ARGM-MNR', 'I-ARGM-MNR', 'O']}], 'words': ['Lukas', 'eats', 'a', 'chicken', 'with', 'hands', '.']}
{'verbs': [{'verb': 'eats', 'description': '[ARG0: Lukas] [V: eats] [ARG1: a chicken with olives] .', 'tags': ['B-ARG0', 'B-V', 'B-ARG1', 'I-ARG1', 'I-ARG1', 'I-ARG1', 'O']}], 'words': ['Lukas', 'eats', 'a', 'chicken', 'with', 'olives', '.']}


In [46]:
s=['Lukas eats a chicken with tomatoes.', 'Lukas eats a chicken with forks.']
for x in s:
    pred=predictor_Bert.predict(x)
    print(pred)

{'verbs': [{'verb': 'eats', 'description': '[ARG0: Lukas] [V: eats] [ARG1: a chicken] [ARG2: with tomatoes] .', 'tags': ['B-ARG0', 'B-V', 'B-ARG1', 'I-ARG1', 'B-ARG2', 'I-ARG2', 'O']}], 'words': ['Lukas', 'eats', 'a', 'chicken', 'with', 'tomatoes', '.']}
{'verbs': [{'verb': 'eats', 'description': '[ARG0: Lukas] [V: eats] [ARG1: a chicken] [ARGM-MNR: with forks] .', 'tags': ['B-ARG0', 'B-V', 'B-ARG1', 'I-ARG1', 'B-ARGM-MNR', 'I-ARGM-MNR', 'O']}], 'words': ['Lukas', 'eats', 'a', 'chicken', 'with', 'forks', '.']}


In [47]:
s=['I drink whisky with Ice', 'I drink wiskey with Jhon']
for x in s:
    pred=predictor_Bert.predict(x)
    print(pred)

{'verbs': [{'verb': 'drink', 'description': '[ARG0: I] [V: drink] [ARG1: whisky] [ARGM-MNR: with Ice]', 'tags': ['B-ARG0', 'B-V', 'B-ARG1', 'B-ARGM-MNR', 'I-ARGM-MNR']}], 'words': ['I', 'drink', 'whisky', 'with', 'Ice']}
{'verbs': [{'verb': 'drink', 'description': '[ARG0: I] [V: drink] [ARG1: wiskey] [ARGM-COM: with Jhon]', 'tags': ['B-ARG0', 'B-V', 'B-ARG1', 'B-ARGM-COM', 'I-ARGM-COM']}], 'words': ['I', 'drink', 'wiskey', 'with', 'Jhon']}


In [55]:
s=['I have breakfast with Coffee', 'I have breakfast with Sunny']
for x in s:
    pred=predictor_Bert.predict(x)
    print(pred)

{'verbs': [{'verb': 'have', 'description': '[ARG0: I] [V: have] [ARG1: breakfast] [ARGM-MNR: with Coffee]', 'tags': ['B-ARG0', 'B-V', 'B-ARG1', 'B-ARGM-MNR', 'I-ARGM-MNR']}], 'words': ['I', 'have', 'breakfast', 'with', 'Coffee']}
{'verbs': [{'verb': 'have', 'description': '[ARG0: I] [V: have] [ARG1: breakfast] [ARGM-COM: with Sunny]', 'tags': ['B-ARG0', 'B-V', 'B-ARG1', 'B-ARGM-COM', 'I-ARGM-COM']}], 'words': ['I', 'have', 'breakfast', 'with', 'Sunny']}


In [57]:
s=['I come by bike', 'I come by noon']
for x in s:
    pred=predictor_Bert.predict(x)
    print(pred)

{'verbs': [{'verb': 'come', 'description': '[ARG1: I] [V: come] [ARGM-MNR: by bike]', 'tags': ['B-ARG1', 'B-V', 'B-ARGM-MNR', 'I-ARGM-MNR']}], 'words': ['I', 'come', 'by', 'bike']}
{'verbs': [{'verb': 'come', 'description': '[ARG1: I] [V: come] [ARGM-TMP: by noon]', 'tags': ['B-ARG1', 'B-V', 'B-ARGM-TMP', 'I-ARGM-TMP']}], 'words': ['I', 'come', 'by', 'noon']}


In [59]:
s=['Lukas saw a man with a knife', 'Lukas saw a man with binoculars']
for x in s:
    pred=predictor_Bert.predict(x)
    print(pred)

{'verbs': [{'verb': 'saw', 'description': '[ARG0: Lukas] [V: saw] [ARG1: a man with a knife]', 'tags': ['B-ARG0', 'B-V', 'B-ARG1', 'I-ARG1', 'I-ARG1', 'I-ARG1', 'I-ARG1']}], 'words': ['Lukas', 'saw', 'a', 'man', 'with', 'a', 'knife']}
{'verbs': [{'verb': 'saw', 'description': '[ARG0: Lukas] [V: saw] [ARG1: a man] [ARGM-MNR: with binoculars]', 'tags': ['B-ARG0', 'B-V', 'B-ARG1', 'I-ARG1', 'B-ARGM-MNR', 'I-ARGM-MNR']}], 'words': ['Lukas', 'saw', 'a', 'man', 'with', 'binoculars']}


In [63]:
s=["Lukas having a beer with a friend.","Lukas having a beer with a lemon."]
for x in s:
    pred=predictor_Bert.predict(x)
    print(pred)

{'verbs': [{'verb': 'having', 'description': '[ARG0: Lukas] [V: having] [ARG1: a beer] [ARGM-COM: with a friend] .', 'tags': ['B-ARG0', 'B-V', 'B-ARG1', 'I-ARG1', 'B-ARGM-COM', 'I-ARGM-COM', 'I-ARGM-COM', 'O']}], 'words': ['Lukas', 'having', 'a', 'beer', 'with', 'a', 'friend', '.']}
{'verbs': [{'verb': 'having', 'description': '[ARG0: Lukas] [V: having] [ARG1: a beer] [ARG2: with a lemon] .', 'tags': ['B-ARG0', 'B-V', 'B-ARG1', 'I-ARG1', 'B-ARG2', 'I-ARG2', 'I-ARG2', 'O']}], 'words': ['Lukas', 'having', 'a', 'beer', 'with', 'a', 'lemon', '.']}


In [64]:
s=["Lukas helps a man with a disability.","Lukas helps a man with a task."]
for x in s:
    pred=predictor_Bert.predict(x)
    print(pred)

{'verbs': [{'verb': 'helps', 'description': '[ARG0: Lukas] [V: helps] [ARG2: a man with a disability] .', 'tags': ['B-ARG0', 'B-V', 'B-ARG2', 'I-ARG2', 'I-ARG2', 'I-ARG2', 'I-ARG2', 'O']}], 'words': ['Lukas', 'helps', 'a', 'man', 'with', 'a', 'disability', '.']}
{'verbs': [{'verb': 'helps', 'description': '[ARG0: Lukas] [V: helps] [ARG2: a man] [ARG1: with a task] .', 'tags': ['B-ARG0', 'B-V', 'B-ARG2', 'I-ARG2', 'B-ARG1', 'I-ARG1', 'I-ARG1', 'O']}], 'words': ['Lukas', 'helps', 'a', 'man', 'with', 'a', 'task', '.']}


## SPAN IDENTIFICATION

### LONG NER

In [5]:
def eval_NER(sentences,predictor):
    fails=0
    golden=['B-ARG0', 'I-ARG0', 'I-ARG0', 'B-V', 'B-ARG1', 'I-ARG1', 'I-ARG1', 'B-ARGM-LOC', 'I-ARGM-LOC']
    for s in sentences:
        pred=predictor.predict(s)
        success=check_NER_tags(pred,golden)
        if not success:
            fails+=1
    return fails/len(sentences)*100

In [45]:

with open(path+"NER_sentences.json","r") as f:
    di=json.load(f)

golden=list(di.keys())[0]
sentences=di[golden]
golden=['B-ARG0', 'I-ARG0', 'I-ARG0', 'B-V', 'B-ARG1', 'I-ARG1', 'I-ARG1', 'B-ARGM-LOC', 'I-ARGM-LOC']
rate=eval_NER(sentences,predictor)
print(f"Failure rate: {rate}. Total number of tests: {len(sentences)}")
append_to_results(results_path,{f"NER_sents":rate})


Failure rate: 67.0. Total number of tests: 100


### LONG SPAN ADJECTIVES

## ROBUSTNESS

### Typos

In [ ]:
for i in range(1,5):#becuase we have 4 files like that

    with open(path+f"sents_{i}_typos.json", 'r') as f:
        sentences = json.load(f)

    sentences=list(sentences.items())

    rate_typos_n=eval_predictor_INV(sentences,predictor,"evaluate_INV_alltags")

    print(f"Rate of failure with {i} typos per sentence: ",rate_typos_n)

    append_to_results(results_path,{f"sents_{i}_typos":rate_typos_n})


## PARAPHRASING

### passive and actrive trasnformarion

In [47]:
with open(path+"activepassive_sentences.json", 'r') as f:
    sentences = json.load(f)

sentences=list(sentences.items())

rate=eval_predictor_INV(sentences,predictor,"evaluate_INV_sameArgs")

print(f"Rate of failure with for INV activepassive ",rate)

append_to_results(results_path,{f"activepassive_sents":rate})


different arguments
different arguments
different arguments
different arguments
different arguments
Rate of failure with for INV activepassive  21.73913043478261


### Questions phrasing
Idea for testing is to take all the arguments from from the verb that is the same(not the auxialiry).
 All the arguments should be the same except for the verb.

In [48]:
with open(path+"question_rephrase.json", 'r') as f:
    sentences = json.load(f)

sentences=list(sentences.items())

rate=eval_predictor_INV(sentences,predictor,"evaluate_INV_sameArgs")

print(f"Rate of failure with for INV in question rephrasing task ",rate)

append_to_results(results_path,{f"question_rephrase":rate})

different arguments
Rate of failure with for INV in question rephrasing task  7.142857142857142


In [49]:
"""
pred1=predictor_Bert.predict("It annoys me the results of the election")
pred1


pred=predictor_Bert.predict("It Is great the results of the election")
pred

pred=predictor_Bert.predict("He was the one that did not want to go to the party")
for v in pred['verbs']:
  print(v['description'])
  

pred=predictor_Bert.predict("she is the sister that likes milk")
for v in pred['verbs']:
  print(v['description'])
  

relative_pronouns = {
    "who": "The person who stole my bike was caught by the police.",
    "whom": "The woman whom I saw at the store was carrying a lot of groceries.",
    "which": "The car which I rented for my vacation was very comfortable.",
    "whose": "The man whose wallet was lost reported the incident to the authorities.",
    "where": "The park where we had our picnic was very peaceful.",
    "when": "The time when we first met is a very special memory for me.",
    "that": "The book that I'm reading right now is very suspenseful.",
    "whoever": "Whoever finishes first gets a prize.",
    "whichever": "You can choose whichever movie you want to see.",
}
for x in relative_pronouns:
  print(x)
  pred=predictor_Bert.predict(relative_pronouns[x])
  for v in pred['verbs']:
    print(v['description'])
  
pred=predictor_Bert.predict("The partner of the lady, who is very old, has exacly my t-shirt")
pred
pred['verbs'][0]['description']

pred=predictor_Bert.predict("The partner of the lady who is very old, has exacly my t-shirt")
pred
pred['verbs'][0]['description']

pred=predictor_Bert.predict("The partner of the lady who is very old has exacly my t-shirt")
pred
pred['verbs'][0]['description']
pred=predictor_Bert.predict("The cars in the shop which is expensive, are exacly what i dream")
pred
pred['verbs'][0]['description']

Ok so the idea is the coreference u dont need here becuase SRL is sentence based very much and for every predicate
you have a sentence. So the coreference is not needed. But sometimes you have that in the same sentence.
"""

'\npred1=predictor_Bert.predict("It annoys me the results of the election")\npred1\n\n\npred=predictor_Bert.predict("It Is great the results of the election")\npred\n\npred=predictor_Bert.predict("He was the one that did not want to go to the party")\nfor v in pred[\'verbs\']:\n  print(v[\'description\'])\n  \n\npred=predictor_Bert.predict("she is the sister that likes milk")\nfor v in pred[\'verbs\']:\n  print(v[\'description\'])\n  \n\nrelative_pronouns = {\n    "who": "The person who stole my bike was caught by the police.",\n    "whom": "The woman whom I saw at the store was carrying a lot of groceries.",\n    "which": "The car which I rented for my vacation was very comfortable.",\n    "whose": "The man whose wallet was lost reported the incident to the authorities.",\n    "where": "The park where we had our picnic was very peaceful.",\n    "when": "The time when we first met is a very special memory for me.",\n    "that": "The book that I\'m reading right now is very suspenseful.

#### PP -ambiguity
 

I went to the resturant by the Hutsin
['[ARG0: I] [V: went] [ARG4: to the resturant by the Hutsin]']



I went to the resturant by bike
['[ARG0: I] [V: went] [ARG4: to the resturant] [ARGM-MNR: by bike]']





# The cimitery of tests... [DONT RUN]





### INCISI

In [194]:
sentences=["This, friend, is epic ","This friend is epic"]
for s in sentences:
  print(s)
  pred=predictor_Bert.predict(s)
  desc=[x['description'] for x in pred['verbs']]
  #print(verbs)
  print(desc)
  print("\n\n")

This, friend, is epic 
['[ARG1: This ,] [ARGM-DIS: friend] , [V: is] [ARG2: epic]']



This friend is epic
['[ARG1: This friend] [V: is] [ARG2: epic]']





In [ ]:
(I might stay, tonight, with you)

### DIfferent frames for labels recognition
With mutiple verbs, argoument confusion 

### Saxon genitive

In [ ]:
paired_sentences = [    ["The president's decision to withdraw troops from the region caused controversy among military leaders.",      "The decision of the president to withdraw troops from the region caused controversy among military leaders."],
    ["The CEO's success in turning around the struggling company was due in large part to her innovative marketing strategies.",      "The success of the CEO in turning around the struggling company was due in large part to her innovative marketing strategies."],
    ["The director's vision for the film was not fully realized due to budget constraints and scheduling conflicts.",      "The vision of the director for the film was not fully realized due to budget constraints and scheduling conflicts."],
    ["The professor's lecture on the history of ancient Rome was attended by a packed auditorium of eager students.",      "The lecture on the history of ancient Rome of the professor was attended by a packed auditorium of eager students."],
    ["The artist's latest work, a sculpture made entirely of recycled materials, was featured in a prominent gallery exhibition.",      "The latest work of the artist, a sculpture made entirely of recycled materials, was featured in a prominent gallery exhibition."],
    ["The athlete's rigorous training regimen and strict diet led to her record-breaking performance at the championship.",      "The rigorous training regimen and strict diet of the athlete led to her record-breaking performance at the championship."],
    ["The author's use of symbolism and metaphor in her novel added depth and complexity to the story.",      "The use of symbolism and metaphor in her novel of the author added depth and complexity to the story."]
]


### Intransitive verbs

Intransitive verbs can be tricky:
So there Unaccusative and Uneragtive are intrasntivie verbs. The difference is that unergative the subject is considered an againt, initating the action. whereas unaccusative the subject is considered the patient of the action.
Naomi worked [argent]
Naomi fell  [theme/patinet]

an MFT test can just intransitive verbs being used as transitive verbs.

and a directionality test can be used to see if the model is able to recognize the difference between unergative and unaccusative verbs. 
like the butter melted - the butter smells


In [ ]:
for s in sentences:
  print(s)
  pred=predictor.predict(s)
  desc=[x['description'] for x in pred['verbs']]
  #print(verbs)
  print(desc)
  print("\n\n")

The sun rises in the east.
['[ARG1: The sun] [V: rises] [ARGM-LOC: in the east] .']



He slept for 10 hours.
['[ARG0: He] [V: slept] [ARGM-TMP: for 10 hours] .']



They danced all night.
['[ARG0: They] [V: danced] [ARGM-TMP: all night] .']



The flowers bloomed in the spring.
['[ARG0: The flowers] [V: bloomed] [ARGM-TMP: in the spring] .']



The baby cried all night.
['[ARG0: The baby] [V: cried] [ARGM-TMP: all night] .']



The cat meowed loudly.
['[ARG0: The cat] [V: meowed] [ARGM-MNR: loudly] .']



The car stopped suddenly.
['[ARG1: The car] [V: stopped] [ARGM-MNR: suddenly] .']



She ran around the park.
['[ARG0: She] [V: ran] [ARGM-DIR: around the park] .']



The plane took off at 9 am.
['[ARG1: The plane] [V: took] off [ARGM-TMP: at 9 am] .']



The wind blew fiercely.
['[ARG1: The wind] [V: blew] [ARGM-MNR: fiercely] .']



The tree swayed in the wind.
['[ARG1: The tree] [V: swayed] [ARGM-LOC: in the wind] .']



The river flows downstream.
['[ARG1: The river] [V: flows] 

In [ ]:
sentences = ['The snow fell gently from the sky.',
             'She laughed uncontrollably at the joke.',
             'The sun set over the horizon.',
             'The children played happily in the park.',
             'The fire burned brightly in the fireplace.',
             'The wind howled outside the window.',
             'The cake baked for an hour.',
             'The door opened slowly.',
             'The crowd cheered loudly at the concert.',
             'The athlete ran around the track.',
             'The tree grew tall and strong.',
             'The river flowed peacefully.',
             'The bird flew gracefully through the air.',
             'The fish swam quickly in the water.',
             'The dog barked loudly at the mailman.',
             'The car drove smoothly down the road.',
             'The plane flew high above the clouds.',
             'The moon shone brightly in the sky.',
             'The stars twinkled in the night sky.',
             'The flowers swayed in the breeze.']


In [ ]:
for s in sentences:
  print(s)
  pred=predictor.predict(s)
  desc=[x['description'] for x in pred['verbs']]
  #print(verbs)
  print(desc)
  print("\n\n")

The snow fell gently from the sky.
['[ARG1: The snow] [V: fell] [ARGM-MNR: gently] [ARG3: from the sky] .']



She laughed uncontrollably at the joke.
['[ARG0: She] [V: laughed] [ARGM-MNR: uncontrollably] [ARG2: at the joke] .']



The sun set over the horizon.
['[ARG1: The sun] [V: set] [ARG2: over the horizon] .']



The children played happily in the park.
['[ARG0: The children] [V: played] [ARGM-MNR: happily] [ARGM-LOC: in the park] .']



The fire burned brightly in the fireplace.
['[ARG1: The fire] [V: burned] [ARGM-MNR: brightly] [ARGM-LOC: in the fireplace] .']



The wind howled outside the window.
['[ARG0: The wind] [V: howled] [ARGM-LOC: outside the window] .']



The cake baked for an hour.
['[ARG1: The cake] [V: baked] [ARGM-TMP: for an hour] .']



The door opened slowly.
['[ARG1: The door] [V: opened] [ARGM-MNR: slowly] .']



The crowd cheered loudly at the concert.
['[ARG0: The crowd] [V: cheered] [ARGM-MNR: loudly] [ARG1: at the concert] .']



The athlete ran around 

In [ ]:
sent="Many people keep falling for this CON - GAME that lower taxes on the rich benefits everyone . "
predictor.predict(sent)

{'verbs': [{'verb': 'keep',
   'description': '[ARG0: Many people] [V: keep] [ARG1: falling for this CON - GAME that lower taxes on the rich benefits everyone] .',
   'tags': ['B-ARG0',
    'I-ARG0',
    'B-V',
    'B-ARG1',
    'I-ARG1',
    'I-ARG1',
    'I-ARG1',
    'I-ARG1',
    'I-ARG1',
    'I-ARG1',
    'I-ARG1',
    'I-ARG1',
    'I-ARG1',
    'I-ARG1',
    'I-ARG1',
    'I-ARG1',
    'I-ARG1',
    'O']},
  {'verb': 'falling',
   'description': '[ARG1: Many people] keep [V: falling] [ARGM-PRP: for this CON - GAME that lower taxes on the rich benefits everyone] .',
   'tags': ['B-ARG1',
    'I-ARG1',
    'O',
    'B-V',
    'B-ARGM-PRP',
    'I-ARGM-PRP',
    'I-ARGM-PRP',
    'I-ARGM-PRP',
    'I-ARGM-PRP',
    'I-ARGM-PRP',
    'I-ARGM-PRP',
    'I-ARGM-PRP',
    'I-ARGM-PRP',
    'I-ARGM-PRP',
    'I-ARGM-PRP',
    'I-ARGM-PRP',
    'I-ARGM-PRP',
    'O']}],
 'words': ['Many',
  'people',
  'keep',
  'falling',
  'for',
  'this',
  'CON',
  '-',
  'GAME',
  'that',
  'lower'

In [ ]:
predictor.predict("He walked the dog to the park and back.")

{'verbs': [{'verb': 'walked',
   'description': '[ARG0: He] [V: walked] [ARG1: the dog] [ARGM-GOL: to the park and back] .',
   'tags': ['B-ARG0',
    'B-V',
    'B-ARG1',
    'I-ARG1',
    'B-ARGM-GOL',
    'I-ARGM-GOL',
    'I-ARGM-GOL',
    'I-ARGM-GOL',
    'I-ARGM-GOL',
    'O']}],
 'words': ['He',
  'walked',
  'the',
  'dog',
  'to',
  'the',
  'park',
  'and',
  'back',
  '.']}

In [ ]:
predictor.predict("He walked with the dog to the park and back.")

{'verbs': [{'verb': 'walked',
   'description': '[ARG0: He] [V: walked] [ARGM-COM: with the dog] [ARGM-GOL: to the park and back] .',
   'tags': ['B-ARG0',
    'B-V',
    'B-ARGM-COM',
    'I-ARGM-COM',
    'I-ARGM-COM',
    'B-ARGM-GOL',
    'I-ARGM-GOL',
    'I-ARGM-GOL',
    'I-ARGM-GOL',
    'I-ARGM-GOL',
    'O']}],
 'words': ['He',
  'walked',
  'with',
  'the',
  'dog',
  'to',
  'the',
  'park',
  'and',
  'back',
  '.']}

In [ ]:
predictor.predict("I see a man with a telescope under his arm")

{'verbs': [{'verb': 'see',
   'description': '[ARG0: I] [V: see] [ARG1: a man with a telescope under his arm]',
   'tags': ['B-ARG0',
    'B-V',
    'B-ARG1',
    'I-ARG1',
    'I-ARG1',
    'I-ARG1',
    'I-ARG1',
    'I-ARG1',
    'I-ARG1',
    'I-ARG1']}],
 'words': ['I',
  'see',
  'a',
  'man',
  'with',
  'a',
  'telescope',
  'under',
  'his',
  'arm']}

In [ ]:
predictor.predict("The plane took a large turn")

{'verbs': [{'verb': 'took',
   'description': '[ARG0: The plane] [V: took] [ARG1: a large turn]',
   'tags': ['B-ARG0', 'I-ARG0', 'B-V', 'B-ARG1', 'I-ARG1', 'I-ARG1']}],
 'words': ['The', 'plane', 'took', 'a', 'large', 'turn']}

### Parenthetical Elements 